# 모델매개변수 최적화 하기
- 매개변수의 오류에 대한 그래디언트를 계산하고 경사하강법을통해 매개변수를 최적화한다

In [87]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)
#데이터와 레이블 확인
training_data.train_data.shape  #torch.Size([60000, 28, 28])
training_data.train_labels.shape  #torch.Size([60000])

# n = 1
# plt.imshow(training_data.train_data[n], cmap='gray')
# print(training_data.train_labels[n]) # 0번 레이블이다 = 상의

# 데이터 로더 만들기
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

# 형태확인 64개의 데이터, 1채널=흑백 이미지는 28*28 크기
next(iter(train_dataloader))[0].size()

torch.Size([64, 1, 28, 28])

In [88]:
train_dataloader.dataset.train_data.shape

torch.Size([60000, 28, 28])

In [89]:
# 신경망 정의 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_leakyrelu_stack = nn.Sequential(
            # 입력 784채널 출력 512채널
            nn.Linear(28*28, 512),
            nn.LeakyReLU(),
            nn.Linear(512,512),
            nn.LeakyReLU(),
            nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_leakyrelu_stack(x)
        return logits

# 모델 인스턴스 생성
model =  NeuralNetwork()
model
    

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_leakyrelu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [90]:
'''hyper params
epoch: 데이터셋 전체를 학습에 반복시키는 횟수
batch_size: 매개변수를 업데이트 하기전 신경망에 전파할 데이터수 
learning_rate: 각 배치/에폭에서 모델의 매개변수를 조절하는 비율이다
    값이작으면 학습속도가 느려지고,크면 예측 할수없는 동작이 발생(우물점프?) 
'''

learning_rate = 1e-3
batch_size = 64
epochs = 5
# 6만개를 64개씩 나누어서 학습을 시키고 
# 6만개를 총 5번 반복해서 학습한다는 뜻이다

# 손실함수 초기화
# 크로스엔트로피: 다중분류일때 사용, 소프트맥스와 nll을 합친것이다
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저는 매개변수를 최적화하는 알고리즘을 설정하는것이다
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

'''
optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정
loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파
변화도 계산후 optimizer.step() 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정
'''


'\noptimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정\nloss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파\n변화도 계산후 optimizer.step() 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정\n'

In [91]:
# 위를 train_loop와 test_loop로 정리하면

def train_loop(dataloader, model, loss_fn, optimizer):
    # dataloader = train_dataloader
    size = len(dataloader.dataset)
    # 배치가 몇번째인지, 한배치의 갯수에 맞게 X,y 각각 여러개
    for batch,(X,y) in enumerate(dataloader):
        # 여기서 예측과 손실을 계산한다
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파 실행
        optimizer.zero_grad()
        loss.backward()  # 그래디언트 계산
        optimizer.step()  # 계산된 그래디언트로 매개변수 조정
        
        if batch%100 ==0:
            loss, current =loss.item(), (batch+1) *len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [92]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304832 [   64/60000]
loss: 2.278584 [ 6464/60000]
loss: 2.273233 [12864/60000]
loss: 2.273821 [19264/60000]
loss: 2.274278 [25664/60000]
loss: 2.231444 [32064/60000]
loss: 2.230968 [38464/60000]
loss: 2.173136 [44864/60000]
loss: 2.195565 [51264/60000]
loss: 2.146554 [57664/60000]
Test Error: 
 Accuracy: 37.0%, Avg loss: 2.155211 

Epoch 2
-------------------------------
loss: 2.168530 [   64/60000]
loss: 2.131942 [ 6464/60000]
loss: 2.097974 [12864/60000]
loss: 2.118147 [19264/60000]
loss: 2.056571 [25664/60000]
loss: 2.023834 [32064/60000]
loss: 2.005002 [38464/60000]
loss: 1.969051 [44864/60000]
loss: 1.923591 [51264/60000]
loss: 1.921978 [57664/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.879166 

Epoch 3
-------------------------------
loss: 1.915569 [   64/60000]
loss: 1.891047 [ 6464/60000]
loss: 1.755988 [12864/60000]
loss: 1.775445 [19264/60000]
loss: 1.781431 [25664/60000]
loss: 1.682112 [32064/60000]
loss: 1.656848 [38464/

In [93]:
# 모델 저장 하기
torch.save(model.state_dict(), 'model_07.pth')